# Extracción de datos desde la API: 
### OBTENCIÓN DE DATOS DE EVENTOS.

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# URL base de la API del Ayuntamiento de Madrid
URL = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

def obtener_eventos():
    headers = {"Accept": "application/json"}  # Encabezados para la solicitud
    response = requests.get(URL, headers=headers)  # Petición GET a la API
    
    if response.status_code == 200:
        datos = response.json()
        return datos
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

In [3]:
data = obtener_eventos()

In [4]:
data2 = data["@graph"]

In [5]:
data.keys()

dict_keys(['@context', '@graph'])

In [40]:
eventos = []
    
    # Iterar sobre la lista de eventos dentro de "@graph"
for evento in data2:
    id_evento = evento.get("id", None)
    nombre_evento = evento.get("title", "")
    url_evento = evento.get("link", "")
    fecha_inicio = evento.get("dtstart", None)
    fecha_fin = evento.get("dtend", None)
    horario = evento.get("time", "")
    organizacion = evento.get("organization", {}).get("organization-name", "")

        # Obtener dirección y código postal
    direccion_info = evento.get("address", {}).get("area", {})
    codigo_postal = direccion_info.get("postal-code", None)
    direccion = direccion_info.get("street-address", "")
        # Convertir fechas al formato DATE de PostgreSQL
    fecha_inicio = datetime.strptime(fecha_inicio, "%Y-%m-%d %H:%M:%S.%f").date() if fecha_inicio else None
    fecha_fin = datetime.strptime(fecha_fin, "%Y-%m-%d %H:%M:%S.%f").date() if fecha_fin else None

    eventos.append((id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion))

In [7]:
eventos = []
    
# Iterar sobre la lista de eventos dentro de "@graph"
for evento in data2:
    fecha_inicio = evento.get("dtstart", "")
    fecha_fin = evento.get("dtend", "")

    # **Filtrar solo eventos entre 2025-03-01 y 2025-03-02**
    if fecha_inicio >= "2025-03-01" and fecha_fin <= "2025-03-02":
        id_evento = evento.get("id", None)
        nombre_evento = evento.get("title", "")
        url_evento = evento.get("link", "")
        horario = evento.get("time", "")
        organizacion = evento.get("organization", {}).get("organization-name", "")

        # Obtener dirección y código postal
        direccion_info = evento.get("address", {}).get("area", {})
        codigo_postal = direccion_info.get("postal-code", None)
        direccion = direccion_info.get("street-address", "")

        eventos.append((id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion))

In [8]:
eventos

[('12682299',
  'Asalto a la lectura. Tres cerditos y más',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=6aec3e3332fe4910VgnVCM1000001d4a900aRCRD',
  '28015',
  'CALLE AMANIEL 29',
  '11:00',
  '2025-03-01 11:00:00.0',
  '2025-03-01 23:59:00.0',
  'Centro Cultural Clara del Rey - Museo ABC (Centro)'),
 ('12691299',
  'Atlantiques + Dahomey',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=3b9464926f415910VgnVCM1000001d4a900aRCRD',
  '28045',
  'PLAZA LEGAZPI 8',
  '19:30',
  '2025-03-01 19:30:00.0',
  '2025-03-01 23:59:00.0',
  'Cineteca Madrid'),
 ('12682147',
  'Baile de Carnaval. Concierto de Origen',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1551fcf4aade4910VgnVCM2000001f4a900aRCRD',
  None,
  '',
  '19:30',
  '2025-03-01 19:30:00.0',
  '2025-03-01 23:59:00.0',
  ''),
 ('12682149',
  'Baile de Carna

In [9]:
df = pd.DataFrame(eventos, columns=["id_evento", "nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "fecha_inicio", "fecha_fin", "organizacion"])

In [10]:
df

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
0,12682299,Asalto a la lectura. Tres cerditos y más,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28015,CALLE AMANIEL 29,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,Centro Cultural Clara del Rey - Museo ABC (Cen...
1,12691299,Atlantiques + Dahomey,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,19:30,2025-03-01 19:30:00.0,2025-03-01 23:59:00.0,Cineteca Madrid
2,12682147,Baile de Carnaval. Concierto de Origen,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,19:30,2025-03-01 19:30:00.0,2025-03-01 23:59:00.0,
3,12682149,Baile de Carnaval. Concierto de Versioneta,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,21:15,2025-03-01 21:15:00.0,2025-03-01 23:59:00.0,
4,12691752,Carnaval en Matadero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,10:30,2025-03-01 10:30:00.0,2025-03-01 23:59:00.0,Matadero Madrid
5,12686562,Centro de Mayores La Platanera. Concurso de Di...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28021,CALLE CONCEPCION DE LA OLIVA 1,21:30,2025-03-01 21:30:00.0,2025-03-01 23:59:00.0,Centro Municipal de Mayores La Platanera
6,12686542,Centro de Mayores Nuestra Señora de los Ángele...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28041,CALLE ARAMA 1,21:30,2025-03-01 21:30:00.0,2025-03-01 23:59:00.0,Centro Municipal de Mayores Nuestra Señora de ...
7,12693374,Chup-Chup,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,CALLE FEBRERO 4,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Centro Cultural José Luis López Vázquez (San B...
8,12690609,Cuatro historias de alquiler,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,CALLE BOLTAÑA 27,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Centro Cultural Buero Vallejo (San Blas - Cani...
9,12039151,De Casa de Campo a Madrid Río I,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28011,PARQUE CASA DE CAMPO (JUNTO AL EMBARCADERO DEL...,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,Centro de Información y Educación Ambiental de...


In [12]:
df.shape

(25, 9)